In [1]:
import os
import acquire
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# import xgboost as xgb
from functools import reduce

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
from env import path

In [3]:
df1 = acquire.read_data('data01.csv')
df2 = acquire.read_data('data02.csv')
df3 = acquire.read_data('data03.csv')
df4 = acquire.read_data('data04.csv')
df5 = acquire.read_data('data05.csv')
df7 = acquire.read_data('data07.csv')

In [4]:
# dfs = [df1, df2, df3]

In [5]:
df1.head()

,CASEID,INVIEWN,B1AGE,AGER,AGERR,B2EDU,HIGHSCHL,B3EMPLY1,B3EMPLY2,B3EMPLY3,...,NAWYEAR,RELAC1,LENGTHC1,C1SITUAT,PARTNERS,PABUSE,SPANISH1,INMONTH,INYEAR,INDAY
0,108395,1,18,1,1,3,0,2,9999,9999,...,99,6,1,1,1,1,2,1,1998,27
1,108397,1,30,3,2,7,1,3,9999,9999,...,99,5,2,1,1,1,2,1,1998,27
2,108399,1,27,3,1,3,0,9,9999,9999,...,99,5,5,1,1,1,2,1,1998,30
3,108443,1,19,1,1,3,0,7,9999,9999,...,99,5,2,1,1,1,2,2,1998,9
4,108444,1,18,1,1,3,0,9,9999,9999,...,99,6,2,1,1,1,2,2,1998,9


In [6]:
list(df1.columns)

['CASEID',
 'INVIEWN',
 'B1AGE',
 'AGER',
 'AGERR',
 'B2EDU',
 'HIGHSCHL',
 'B3EMPLY1',
 'B3EMPLY2',
 'B3EMPLY3',
 'B3EMPLY4',
 'EMPLOYR',
 'UNEMPLOY',
 'HOMEMAKR',
 'B3HOURS',
 'B3VHOUR',
 'B3STUDNT',
 'B4CHANGE',
 'B5CODE1',
 'B5DATE1',
 'B5CODE2',
 'B5DATE2',
 'B5CODE3',
 'B5DATE3',
 'B6RACE',
 'RACER',
 'B6HSPNIC',
 'B6OTHER',
 'B7BIRTH',
 'B7PLACE',
 'B7PLACER',
 'F14CHICG',
 'B8MARITL',
 'MARRIED',
 'SINGLE',
 'SCRSTAT',
 'SCRSTATR',
 'FINSTAT',
 'FINSTATR',
 'ABUSED',
 'CHNGSTAT',
 'CHGTONAW',
 'NAWYEAR',
 'RELAC1',
 'LENGTHC1',
 'C1SITUAT',
 'PARTNERS',
 'PABUSE',
 'SPANISH1',
 'INMONTH',
 'INYEAR',
 'INDAY']

In [7]:
keep1 = ['CASEID', 'ABUSED', 'SCRSTATR', 'LENGTHC1','C1SITUAT','PABUSE']

In [8]:
drop1 = []

In [9]:
for element in list(df1.columns):
    if element not in keep1:
        drop1.append(element)
drop1

['INVIEWN',
 'B1AGE',
 'AGER',
 'AGERR',
 'B2EDU',
 'HIGHSCHL',
 'B3EMPLY1',
 'B3EMPLY2',
 'B3EMPLY3',
 'B3EMPLY4',
 'EMPLOYR',
 'UNEMPLOY',
 'HOMEMAKR',
 'B3HOURS',
 'B3VHOUR',
 'B3STUDNT',
 'B4CHANGE',
 'B5CODE1',
 'B5DATE1',
 'B5CODE2',
 'B5DATE2',
 'B5CODE3',
 'B5DATE3',
 'B6RACE',
 'RACER',
 'B6HSPNIC',
 'B6OTHER',
 'B7BIRTH',
 'B7PLACE',
 'B7PLACER',
 'F14CHICG',
 'B8MARITL',
 'MARRIED',
 'SINGLE',
 'SCRSTAT',
 'FINSTAT',
 'FINSTATR',
 'CHNGSTAT',
 'CHGTONAW',
 'NAWYEAR',
 'RELAC1',
 'PARTNERS',
 'SPANISH1',
 'INMONTH',
 'INYEAR',
 'INDAY']

In [10]:
df1 = df1.drop(columns=drop1, axis=1)

In [11]:
df1

,CASEID,SCRSTATR,ABUSED,LENGTHC1,C1SITUAT,PABUSE
0,108395,1,1,1,1,1
1,108397,1,1,2,1,1
2,108399,1,1,5,1,1
3,108443,1,1,2,1,1
4,108444,1,1,2,1,1
5,108456,1,1,2,3,1
6,108460,1,1,3,1,1
7,108525,1,1,4,4,3
8,108562,1,1,5,1,1
9,109304,3,2,5,9,9


In [12]:
df2.head()

,CASEID,INVIEWN,ABUSED,RACER,D1ALONE,NOHOME,RCHANY,RCHILD17,RCHILD18,DECEASED,...,D7SEX10,D7YR10,D7MTH10,D8JNLFT,D9CODE1,D9CODE2,D9CODE3,D9DATE1,D9DATE2,D9DATE3
0,108395,1,1,1,2,2,1,1,0,0,...,9999,9999,9999,2,9999,9999,9999,1,1,1
1,108397,1,1,9,2,2,0,0,0,0,...,9999,9999,9999,1,4,9999,9999,11,1,1
2,108399,1,1,1,2,2,1,6,0,0,...,9999,9999,9999,2,9999,9999,9999,1,1,1
3,108443,1,1,1,2,2,0,0,0,0,...,9999,9999,9999,2,9999,9999,9999,1,1,1
4,108444,1,1,1,2,2,1,1,0,0,...,9999,9999,9999,2,9999,9999,9999,1,1,1


In [13]:
keep2 = ['D3RCHILT']

In [14]:
df2 = df2[['CASEID','D3RCHILT']]

In [15]:
df2

,CASEID,D3RCHILT
0,108395,1
1,108397,0
2,108399,6
3,108443,0
4,108444,1
5,108456,1
6,108460,1
7,108525,2
8,108562,3
9,109304,0


In [16]:
df3.head()

,CASEID,INVIEWN,ABUSED,HINCOME,E1HEALTH,POORHTH,E2HLTHNW,E3VISIT,E4PROBLM,E5CODE1,...,ANYPINC,F17HINCM,DEPTOT,DEPTOT2,ACCESS,ACCESS2,EMERHELP,SUPPORT,TOTSUPRT,TOTSUPT2
0,108395,1,1,2,3,0,1,5,2,9999,...,1,2,0,0,2,3,4,3,9,10
1,108397,1,1,3,3,0,1,5,1,17,...,1,3,0,0,2,2,4,5,11,11
2,108399,1,1,1,4,0,4,4,2,9999,...,1,1,2,2,2,3,2,5,9,10
3,108443,1,1,2,1,0,1,5,1,4,...,1,2,2,2,3,3,4,5,12,12
4,108444,1,1,888,3,0,4,3,2,9999,...,1,888,1,1,3,3,4,4,11,11


In [17]:
df3 = df3[['CASEID','E13PRGNT','N7PREGNT','TOTSUPRT']]

In [18]:
df3

,CASEID,E13PRGNT,N7PREGNT,TOTSUPRT
0,108395,3,2,9
1,108397,3,3,11
2,108399,3,2,9
3,108443,1,2,12
4,108444,2,2,11
5,108456,3,2,2
6,108460,888,1,10
7,108525,3,2,11
8,108562,3,2,11
9,109304,3,9,12


In [19]:
df3.to_csv("1st_half.csv", index=False)